# Data Prep and Loading

In [1]:
import opendp.prelude as dp
dp.enable_features('contrib')
import numpy as np 
import pandas as pd
from race_imputation import race_imputation_model
import data_preprocessing_utils
# force all columns to show on head call
pd.set_option('display.max_columns', None)


In [3]:
voter_data_all = pd.read_csv('./data/hmda_data_nc.csv', nrows=100)

voter_data_all.head()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2023,549300JOT0D4J0SZIK67,40580,NC,37065,37065020800,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Available,Race Not Available,Sex Not Available,6,2,2,2,1,1,2,2,2,155000.0,NaN,6.750,NaN,2,10359.06,NaN,4108.37,NaN,NaN,360,NaN,NaN,2,2,2,2,155000.0,2,1,1,1,1,NaN,NaN,NaN,9,9,4,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,3,3,7,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,3,3,4,4,3,3,8888,8888,NaN,NaN,3,3,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,3115,53.19,77700,83.83,1053,1654,29
1,2023,549300JOT0D4J0SZIK67,24660,NC,37157,37157041100,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Available,Race Not Available,Sex Not Available,6,2,2,2,32,1,2,2,2,145000.0,NaN,5.500,NaN,2,12136.14,NaN,7284.94,5694.94,NaN,360,NaN,NaN,2,2,2,2,195000.0,2,1,1,1,1,NaN,NaN,NaN,9,9,4,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,3,3,7,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,3,3,4,4,3,3,8888,8888,NaN,NaN,3,3,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,6485,20.20,81500,102.28,2211,2935,41
2,2023,549300JOT0D4J0SZIK67,16740,NC,37025,37025042300,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,6,2,2,2,1,1,2,2,2,265000.0,NaN,6.125,NaN,2,11368.44,NaN,3560.84,2265.84,17.0,360,NaN,NaN,2,2,2,2,265000.0,1,1,3,5,1,NaN,NaN,NaN,9,9,4,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,3,3,7,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,3,3,4,4,3,3,8888,8888,NaN,NaN,3,3,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,4922,44.72,98700,63.83,878,1671,51
3,2023,549300JOT0D4J0SZIK67,99999,NC,37039,37039930502,C,VA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Available,Race Not Available,Sex Not Available,6,2,2,3,1,1,2,2,2,175000.0,NaN,7.125,NaN,2,575.00,NaN,0.00,2406.25,NaN,360,NaN,NaN,2,2,2,2,175000.0,2,1,1,1,1,NaN,NaN,NaN,9,9,4,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,3,3,7,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,3,3,4,4,3,3,8888,8888,NaN,NaN,3,3,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,3587,12.82,70400,102.84,1325,2312,26
4,2023,549300JOT0D4J0SZIK67,11700,NC,37021,37021003203,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Availabl

In [ ]:
voter_data = prep